In [4]:
import torch
import torch.nn as nn
import torchvision
import wandb
from torchsummary import summary
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class CNN(nn.Module):
    def __init__(self, num_filters=32, size_filters=3, activation_func='relu', filter_org=1, num_dense=128, batch_normalisation=False, dropout_rate=0.2, input_channels=3, num_classes=10, num_conv=5):
        '''
        num_filters: Number of filters in each layer --> 32,64,etc
        size_filters: Size of each filter (=F) --> 5,10,etc
        activation_func: Activation function for the convolutional layers --> ReLU, GeLU,SiLU, Mish
        filter_org: Ratio of number of filters in i+1th layer to number of filters in ith layer --> 1,0.5,2,etc
        num_dense: Number of neurons in dense layer --> 128
        batch_normalisation: Whether or not to apply batch normalisation after convolution layers --> True, False
        dropout_rate: Fraction of neurons to randomly drop (=p) --> 0.2 to 0.5
        input_channels: number of channels in input layer --> 3 (RGB)
        num_classes: Number of Classes in the iNaturalist Dataset --> 10
        num_conv: number of Conv-activation-maxpool blocks in the CNN model --> given:5
        '''
        super(CNN, self).__init__()
        self.layers=nn.ModuleList()

        def get_activation(name):
            if name == 'relu':
                return nn.ReLU()
            elif name == 'gelu':
                return nn.GELU()
            elif name == 'silu':
                return nn.SiLU()
            elif name == 'mish':
                return nn.Mish()

        for layer in range(num_conv):
            out_channels=int(num_filters*((filter_org)**(layer)))
            self.layers.append(nn.Conv2d(in_channels=input_channels, out_channels=out_channels, kernel_size=size_filters, padding=size_filters//2))
            if batch_normalisation==True:
                self.layers.append(nn.BatchNorm2d(out_channels))
            input_channels=out_channels
            self.layers.append(get_activation(name=activation_func))    
            self.layers.append(nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        if batch_normalisation==True:
            self.fc_layers = nn.Sequential(nn.Linear(input_channels, num_dense),get_activation(name=activation_func), nn.BatchNorm1d(num_dense), nn.Dropout(p=dropout_rate), nn.Linear(num_dense, num_classes))
        elif batch_normalisation==False:
            self.fc_layers = nn.Sequential(nn.Linear(input_channels, num_dense),get_activation(name=activation_func), nn.Dropout(p=dropout_rate), nn.Linear(num_dense, num_classes))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

my_model = CNN(
    num_filters=32,
    size_filters=3,
    activation_func='relu',
    filter_org=2,
    num_dense=128
).to(device)

summary(my_model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
         MaxPool2d-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 64, 112, 112]          18,496
              ReLU-5         [-1, 64, 112, 112]               0
         MaxPool2d-6           [-1, 64, 56, 56]               0
            Conv2d-7          [-1, 128, 56, 56]          73,856
              ReLU-8          [-1, 128, 56, 56]               0
         MaxPool2d-9          [-1, 128, 28, 28]               0
           Conv2d-10          [-1, 256, 28, 28]         295,168
             ReLU-11          [-1, 256, 28, 28]               0
        MaxPool2d-12          [-1, 256, 14, 14]               0
           Conv2d-13          [-1, 512, 14, 14]       1,180,160
             ReLU-14          [-1, 512,

In [7]:
def get_dataloaders(dir='/kaggle/input/nature-12k/inaturalist_12K/train',augment='No',split=0.2,batch_size=64):
    labels = datasets.ImageFolder(root=dir).targets
    
    val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])
    if augment=='Yes':
        train_transforms = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2,
                                saturation=0.2, hue=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
        ])  
    elif augment=='No':
        train_transforms=val_transforms

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=split, random_state=42)
    train_idx, val_idx = next(splitter.split(torch.zeros(len(labels)), labels))

    train_dataset=datasets.ImageFolder(root=dir,transform=train_transforms)
    val_dataset=datasets.ImageFolder(root=dir,transform=val_transforms)

    train_dataset = Subset(train_dataset, train_idx)
    val_dataset = Subset(val_dataset, val_idx)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader,val_loader

In [8]:
def get_optimizer(optim,lr,model):
    if optim=='sgd':
        return (torch.optim.SGD(model.parameters(), lr, weight_decay=0, momentum=0))
    elif optim=='momentum':
        return (torch.optim.SGD(model.parameters(), lr, weight_decay=0, momentum=0.9))
    elif optim=='adam':
        return (torch.optim.Adam(model.parameters(), lr, weight_decay=0.005))

In [15]:
import wandb
# wandb.init(
#     project="DA6401_Assign2",
# )
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'num_filters':{
            'values':[32,64]
        },
        'size_filters':{
            'values':[3,5,10]
        },
        'activation_func':{
            'values':['relu','gelu','silu','mish']
        },
        'filter_org':{
            'values':[1,0.5,2]
        },
        'num_dense':{
            'values':[128,256,512]
        },
        'batch_size':{
            'values':[16,32,64]
        },
        'optimizer':{
            'values':['sgd','momentum','adam']
        },
        'learning_rate':{
            'min':0.0001,
            'max':0.01
        },
        'data_augmentation':{
            'values':['Yes','No']
        },
        'batch_normalisation':{
            'values':[True,False]
        },
        'dropout_rate':{
            'min':0.2,
            'max':0.5
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="DA6401_Assign2", entity="ishita49-indian-institute-of-technology-madras")

Create sweep with ID: ix36j15i
Sweep URL: https://wandb.ai/ishita49-indian-institute-of-technology-madras/DA6401_Assign2/sweeps/ix36j15i


In [16]:
def train():
    config_defaults = {
        'num_filters':32,
        'size_filters':3,
        'activation_func':'relu',
        'filter_org':2,
        'num_dense':128,
        'batch_size':64,
        'optimizer':'sgd',
        'learning_rate':0.005,
        'data_augmentation':'No',
        'batch_normalisation':False,
        'dropout_rate':0.2
    }
    wandb.login(key='70a00ae1607c730fb9cd50b1268b191bec7a2901')
    wandb.init(config=config_defaults)
    config = wandb.config

    model = CNN(num_filters=config.num_filters,
                size_filters=config.size_filters,
                activation_func=config.activation_func,
                filter_org=config.filter_org,
                num_dense=config.num_dense,
                batch_normalisation=config.batch_normalisation,
                dropout_rate=config.dropout_rate,
                num_classes=10)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer=get_optimizer(optim=config.optimizer,lr=config.learning_rate,model=model)
    train_loader,val_loader=get_dataloaders(batch_size=config.batch_size,augment=config.data_augmentation)
    # Train
    num_epochs=20
    print("training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss=0
        for i, (images, labels) in enumerate(train_loader):  
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss+=loss.item()
        avg_loss=total_loss/len(train_loader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))
        wandb.log({"epoch": epoch + 1, "train_loss": avg_loss})

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Validation Accuracy: {:.2f}%'.format(accuracy))

    wandb.log({"val_accuracy": accuracy})
    wandb.finish()

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 9264ulfs with config:
wandb: 	activation_func: mish
wandb: 	batch_normalisation: False
wandb: 	batch_size: 16
wandb: 	data_augmentation: Yes
wandb: 	dropout_rate: 0.48513216376589935
wandb: 	filter_org: 1
wandb: 	learning_rate: 0.0026587824562307612
wandb: 	num_dense: 128
wandb: 	num_filters: 32
wandb: 	optimizer: sgd
wandb: 	size_filters: 3
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Using 2 GPUs
training...
Epoch [1/20], Loss: 2.3055
Epoch [2/20], Loss: 2.3044
Epoch [3/20], Loss: 2.3038
Epoch [4/20], Loss: 2.3042
Epoch [5/20], Loss: 2.3039
Epoch [6/20], Loss: 2.3035
Epoch [7/20], Loss: 2.3035
Epoch [8/20], Loss: 2.3034
Epoch [9/20], Loss: 2.3036
Epoch [10/20], Loss: 2.3042
Epoch [11/20], Loss: 2.3033
Epoch [12/20], Loss: 2.3039
Epoch [13/20], Loss: 2.3030
Epoch [14/20], Loss: 2.3035
Epoch [15/20], Loss: 2.3029
Epoch [16/20], Loss: 2.3035
Epoch [17/20], Loss: 2.3035
Epoch [18/20], Loss: 2.3033
Epoch [19/20], Loss: 2.3031
Epoch [20/20], Loss: 2.3034
Validation Accuracy: 10.00%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▅▄▅▄▃▃▂▃▄▂▄▁▃▁▃▃▂▂▂
val_accuracy,▁
epoch,20
train_loss,2.3034
val_accuracy,10


wandb: Agent Starting Run: f1on36qj with config:
wandb: 	activation_func: gelu
wandb: 	batch_normalisation: True
wandb: 	batch_size: 32
wandb: 	data_augmentation: No
wandb: 	dropout_rate: 0.41369682254612383
wandb: 	filter_org: 1
wandb: 	learning_rate: 0.0014733199120718148
wandb: 	num_dense: 128
wandb: 	num_filters: 64
wandb: 	optimizer: adam
wandb: 	size_filters: 10
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Using 2 GPUs
training...
Epoch [1/20], Loss: 2.2738
Epoch [2/20], Loss: 2.2108
Epoch [3/20], Loss: 2.1828
Epoch [4/20], Loss: 2.1684
Epoch [5/20], Loss: 2.1780
Epoch [6/20], Loss: 2.1681
Epoch [7/20], Loss: 2.1746
Epoch [8/20], Loss: 2.1688
Epoch [9/20], Loss: 2.1617
Epoch [10/20], Loss: 2.1486
Epoch [11/20], Loss: 2.1499
Epoch [12/20], Loss: 2.1363
Epoch [13/20], Loss: 2.1274
Epoch [14/20], Loss: 2.1199
Epoch [15/20], Loss: 2.1184
Epoch [16/20], Loss: 2.1009
Epoch [17/20], Loss: 2.1002
Epoch [18/20], Loss: 2.0962
Epoch [19/20], Loss: 2.0869
Epoch [20/20], Loss: 2.0851
Validation Accuracy: 23.85%


...

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,▇▃▂▅▄▃▃▂▃█▅▂▂▅▅▃▆▅▁▁
val_accuracy,▁
epoch,20
train_loss,2.30592
val_accuracy,10


wandb: Agent Starting Run: da5m7oed with config:
wandb: 	activation_func: silu
wandb: 	batch_normalisation: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: No
wandb: 	dropout_rate: 0.3957371450977386
wandb: 	filter_org: 0.5
wandb: 	learning_rate: 0.0019847665770792154
wandb: 	num_dense: 512
wandb: 	num_filters: 32
wandb: 	optimizer: sgd
wandb: 	size_filters: 5
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Using 2 GPUs
training...
Epoch [1/20], Loss: 2.3100
Epoch [2/20], Loss: 2.3101
Epoch [3/20], Loss: 2.3083
Epoch [4/20], Loss: 2.3077
Epoch [5/20], Loss: 2.3079
Epoch [6/20], Loss: 2.3050
Epoch [7/20], Loss: 2.3061
Epoch [8/20], Loss: 2.3068
Epoch [9/20], Loss: 2.3067
Epoch [10/20], Loss: 2.3063
Epoch [11/20], Loss: 2.3069
Epoch [12/20], Loss: 2.3069
Epoch [13/20], Loss: 2.3082
Epoch [14/20], Loss: 2.3061
Epoch [15/20], Loss: 2.3064
Epoch [16/20], Loss: 2.3051
Epoch [17/20], Loss: 2.3081
Epoch [18/20], Loss: 2.3073
Epoch [19/20], Loss: 2.3081
Epoch [20/20], Loss: 2.3062
Validation Accuracy: 10.10%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,██▆▅▅▁▂▃▃▃▄▄▅▃▃▁▅▄▅▃
val_accuracy,▁
epoch,20
train_loss,2.30621
val_accuracy,10.1


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1xgq2v17 with config:
wandb: 	activation_func: gelu
wandb: 	batch_normalisation: False
wandb: 	batch_size: 32
wandb: 	data_augmentation: No
wandb: 	dropout_rate: 0.27130655808110793
wandb: 	filter_org: 1
wandb: 	learning_rate: 0.005722038133631561
wandb: 	num_dense: 256
wandb: 	num_filters: 32
wandb: 	optimizer: adam
wandb: 	size_filters: 3
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
model=CNN(num_filters=64, size_filters=3, activation_func='relu', filter_org=1, num_dense=256, batch_normalisation=True, dropout_rate=0.20426922413644705, input_channels=3, num_classes=10, num_conv=5)

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer=get_optimizer(optim='momentum',lr=0.0032148335356218384,model=model)
train_loader,val_loader=get_dataloaders(batch_size=32,augment='No')
# Train
num_epochs=20
print("training...")
for epoch in range(num_epochs):
    model.train()
    total_loss=0
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    avg_loss=total_loss/len(train_loader)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))

# Validation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Train Accuracy: {:.2f}%'.format(accuracy))

# Validation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Validation Accuracy: {:.2f}%'.format(accuracy))

Using 2 GPUs
training...
Epoch [1/20], Loss: 2.1810
Epoch [2/20], Loss: 2.1016
Epoch [3/20], Loss: 2.0460
Epoch [4/20], Loss: 2.0225
Epoch [5/20], Loss: 1.9799
Epoch [6/20], Loss: 1.9564
Epoch [7/20], Loss: 1.9148
Epoch [8/20], Loss: 1.8996
Epoch [9/20], Loss: 1.8807
Epoch [10/20], Loss: 1.8636
Epoch [11/20], Loss: 1.8316
Epoch [12/20], Loss: 1.8206
Epoch [13/20], Loss: 1.8069
Epoch [14/20], Loss: 1.7896
Epoch [15/20], Loss: 1.7703
Epoch [16/20], Loss: 1.7705
Epoch [17/20], Loss: 1.7631
Epoch [18/20], Loss: 1.7339
Epoch [19/20], Loss: 1.7219
Epoch [20/20], Loss: 1.7062
Train Accuracy: 41.27%
Validation Accuracy: 36.40%


In [8]:
def test_dataloader(dir='/kaggle/input/nature-12k/inaturalist_12K/val',batch_size=32):
    labels = datasets.ImageFolder(root=dir).targets
    
    all_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])
    
    test_dataset=datasets.ImageFolder(root=dir,transform=all_transforms)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2)

    return test_loader

test_loader=test_dataloader(batch_size=32)

# Test
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Test Accuracy: {:.2f}%'.format(accuracy))

Test Accuracy: 37.10%


In [20]:
from collections import Counter

def get_class_counts(dataloader):
    dataset = dataloader.dataset
    targets = [dataset.dataset.targets[i] for i in dataset.indices]
    total=0
    for cls,count in sorted(Counter(targets).items()):
        print(f'Class{cls}: {count} samples')
        total+=count
    print(f'total samples={total}')
    return total

print('No. of samples in each class in training data:')
train_count = get_class_counts(train_loader)
print('\nNo. of samples in each class in Validation data:')
val_count = get_class_counts(val_loader)

print('\nPercentage of train data kept aside for validation={:.2f}%'.format(val_counts*100/(train_counts+val_counts)))

No. of samples in each class in training data:
Class0: 800 samples
Class1: 800 samples
Class2: 800 samples
Class3: 800 samples
Class4: 799 samples
Class5: 800 samples
Class6: 800 samples
Class7: 800 samples
Class8: 800 samples
Class9: 800 samples
total samples=7999

No. of samples in each class in Validation data:
Class0: 200 samples
Class1: 200 samples
Class2: 200 samples
Class3: 200 samples
Class4: 200 samples
Class5: 200 samples
Class6: 200 samples
Class7: 200 samples
Class8: 200 samples
Class9: 200 samples
total samples=2000

Percentage of train data kept aside for validation=20.00%


In [21]:
model=CNN(num_filters=64, size_filters=10, activation_func='relu', filter_org=2, num_dense=512, batch_normalisation=True, dropout_rate=0.35298806906663843, input_channels=3, num_classes=10, num_conv=5)

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer=get_optimizer(optim='sgd',lr=0.0006349956446405355,model=model)
train_loader,val_loader=get_dataloaders(batch_size=32,augment='No')
# Train
num_epochs=20
print("training...")
for epoch in range(num_epochs):
    model.train()
    total_loss=0
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    avg_loss=total_loss/len(train_loader)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))

# Validation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Train Accuracy: {:.2f}%'.format(accuracy))

# Validation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Validation Accuracy: {:.2f}%'.format(accuracy))

test_loader=test_dataloader(batch_size=32)

# Test
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Test Accuracy: {:.2f}%'.format(accuracy))

Using 2 GPUs
training...
Epoch [1/20], Loss: 2.1881
Epoch [2/20], Loss: 2.0808
Epoch [3/20], Loss: 2.0267
Epoch [4/20], Loss: 1.9886
Epoch [5/20], Loss: 1.9478
Epoch [6/20], Loss: 1.9158
Epoch [7/20], Loss: 1.8863
Epoch [8/20], Loss: 1.8508
Epoch [9/20], Loss: 1.8322
Epoch [10/20], Loss: 1.7887
Epoch [11/20], Loss: 1.7574
Epoch [12/20], Loss: 1.7214
Epoch [13/20], Loss: 1.6999
Epoch [14/20], Loss: 1.6507
Epoch [15/20], Loss: 1.5966
Epoch [16/20], Loss: 1.5685
Epoch [17/20], Loss: 1.5072
Epoch [18/20], Loss: 1.4440
Epoch [19/20], Loss: 1.3559
Epoch [20/20], Loss: 1.2720
Train Accuracy: 43.46%
Validation Accuracy: 28.25%
Test Accuracy: 28.95%


In [10]:
import wandb
wandb.login(key='70a00ae1607c730fb9cd50b1268b191bec7a2901')
# wandb.init(
#     project="DA6401_Assign2",
# )
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 3
    },
    'parameters': {
        'num_filters':{
            'values':[64,128]
        },
        'size_filters':{
            'values':[3,5,10]
        },
        'activation_func':{
            'values':['relu']
        },
        'filter_org':{
            'values':[1,0.5,2]
        },
        'num_dense':{
            'values':[128,256,512]
        },
        'batch_size':{
            'values':[16,32]
        },
        'optimizer':{
            'values':['momentum']
        },
        'learning_rate':{
            'min':0.001,
            'max':0.01
        },
        'data_augmentation':{
            'values':['Yes','No']
        },
        'batch_normalisation':{
            'values':[True]
        },
        'dropout_rate':{
            'min':0.2,
            'max':0.4
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="DA6401_Assign2", entity="ishita49-indian-institute-of-technology-madras")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ishita49 (ishita49-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Create sweep with ID: hyhjr9sg
Sweep URL: https://wandb.ai/ishita49-indian-institute-of-technology-madras/DA6401_Assign2/sweeps/hyhjr9sg


In [11]:
from tqdm import tqdm

def train():
    config_defaults = {
        'num_filters':64,
        'size_filters':3,
        'activation_func':'relu',
        'filter_org':2,
        'num_dense':128,
        'batch_size':32,
        'optimizer':'momentum',
        'learning_rate':0.002,
        'data_augmentation':'No',
        'batch_normalisation':True,
        'dropout_rate':0.25
    }
    wandb.init(config=config_defaults, tags="Updated")
    config = wandb.config

    model = CNN(num_filters=config.num_filters,
                size_filters=config.size_filters,
                activation_func=config.activation_func,
                filter_org=config.filter_org,
                num_dense=config.num_dense,
                batch_normalisation=config.batch_normalisation,
                dropout_rate=config.dropout_rate,
                num_classes=10)
    
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer=get_optimizer(optim=config.optimizer,lr=config.learning_rate,model=model)
    train_loader,val_loader=get_dataloaders(batch_size=config.batch_size,augment=config.data_augmentation)
    # Train
    num_epochs=20
    print("training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss=0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss+=loss.item()
        avg_loss=total_loss/len(train_loader)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))
        wandb.log({"epoch": epoch + 1, "train_loss": avg_loss})
    
        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        accuracy = 100 * correct / total
        print('Validation Accuracy: {:.2f}%'.format(accuracy))
    
        wandb.log({"val_accuracy": accuracy})
    wandb.finish()

In [12]:
wandb.agent(sweep_id, function=train,count=15)

wandb: Agent Starting Run: a38mwtom with config:
wandb: 	activation_func: relu
wandb: 	batch_normalisation: True
wandb: 	batch_size: 16
wandb: 	data_augmentation: No
wandb: 	dropout_rate: 0.29529864039607623
wandb: 	filter_org: 1
wandb: 	learning_rate: 0.004681688280367698
wandb: 	num_dense: 512
wandb: 	num_filters: 128
wandb: 	optimizer: momentum
wandb: 	size_filters: 5
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Using 2 GPUs
training...


Epoch [1/20], Loss: 2.5501


Validation Accuracy: 14.45%


Epoch [2/20], Loss: 2.4658


...

Epoch [9/20], Loss: 2.0814


Validation Accuracy: 30.55%


Epoch [10/20], Loss: 2.0570


Validation Accuracy: 28.75%


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▆▄▄▄▃▃▃▂▁
val_accuracy,▁▁▃▅▄▆▄▅█▆
epoch,10
train_loss,2.05702
val_accuracy,28.75


In [1]:
References={
    "CNN Model":"https://www.digitalocean.com/community/tutorials/writing-cnns-from-scratch-in-pytorch",
    "Batch Normalisation":"https://datahacker.rs/017-pytorch-how-to-apply-batch-normalization-in-pytorch/",
    "Dropout":"https://medium.com/@vishnuam/dropout-in-convolutional-neural-networks-cnn-422a4a17da41"
 }